In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# TOMA DETALLE DE DESPACHOS PERO NO SE QUE DEVUELVE
ruta = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/data/detalle_de_despachos.xlsx'
df_despacho = pd.read_excel (ruta)
df_despacho.head()

,FECHA,SOCIO,SACAR,CONTROLADO,FECHA DE \nDESPACHO,CANTIDAD DE\nPALLETS SACADOS,CANTIDAD DE PALLETS A CARGAR,CARGADO,TURNO,OBSERVACIONES
0,2024-12-30 00:00:00,39,X,MIX,2025-01-02,17.0,8.0,SI,SI,NaN
1,2024-12-31 00:00:00,68,X,SANTIAGO/MART,2025-01-02,18.0,12.0,SI,SI,NaN
2,2024-12-31 00:00:00,11,X,MIX,2025-01-02,NaN,10.0,SI,SI,NaN
3,2024-12-30 00:00:00,59,X,CRISTIAN,2025-01-02,19.0,12.0,SI,SI,NaN
4,2024-12-31 00:00:00,12,X,CRISTIAN,2025-01-03,13.0,12.0,SI,SI,NaN


In [13]:
# Elimina espacios y normaliza (ej: 'Estado    ' -> 'Estado')
df_despacho.columns = df_despacho.columns.str.strip()  # Quita espacios
# Opcional: convertir a minúsculas (para uniformidad)
df_despacho.columns = df_despacho.columns.str.lower()  # 'Estado' -> 'estado'
print("Columnas disponibles:", df_despacho.columns.tolist())

Columnas disponibles: ['fecha', 'socio', 'sacar', 'controlado', 'fecha de \ndespacho', 'cantidad de\npallets sacados', 'cantidad de pallets a cargar', 'cargado', 'turno', 'observaciones']


In [14]:
df_despacho.rename(columns={'cantidad de\npallets sacados': 'cantidad_pallets_sacados', 'fecha de \ndespacho': 'fecha_despacho', 'cantidad de\npallets sacados': 'cantidad_pallets_sacados', 'cantidad de pallets a cargar': 'cantidad_pallets_a_cargar'}, inplace=True)

In [15]:
df_despacho.rename(columns={'socio': 'id_socio'}, inplace=True)
print("Columnas disponibles:", df_despacho.columns.tolist())

Columnas disponibles: ['fecha', 'id_socio', 'sacar', 'controlado', 'fecha_despacho', 'cantidad_pallets_sacados', 'cantidad_pallets_a_cargar', 'cargado', 'turno', 'observaciones']


In [16]:
df_despacho["fecha_despacho"] = pd.to_datetime(df_despacho["fecha_despacho"], format="%d/%m/%Y", errors='coerce')

In [17]:
df_despacho.head()

,fecha,id_socio,sacar,controlado,fecha_despacho,cantidad_pallets_sacados,cantidad_pallets_a_cargar,cargado,turno,observaciones
0,2024-12-30 00:00:00,39,X,MIX,2025-01-02,17.0,8.0,SI,SI,NaN
1,2024-12-31 00:00:00,68,X,SANTIAGO/MART,2025-01-02,18.0,12.0,SI,SI,NaN
2,2024-12-31 00:00:00,11,X,MIX,2025-01-02,NaN,10.0,SI,SI,NaN
3,2024-12-30 00:00:00,59,X,CRISTIAN,2025-01-02,19.0,12.0,SI,SI,NaN
4,2024-12-31 00:00:00,12,X,CRISTIAN,2025-01-03,13.0,12.0,SI,SI,NaN


In [18]:
df_despacho['cantidad_pallets_sacados'] = df_despacho['cantidad_pallets_sacados'].fillna(0).astype(int)

In [19]:
# Crear un diccionario para mapear los socios combinados
map_socio = {
    11: '11.111', 111: '11.111',
    17: '17.39', 39: '17.39',
    25: '25.63', 63: '25.63'
}

# Reemplazar los valores de id_socio según el mapeo
df_despacho['id_socio'] = df_despacho['id_socio'].replace(map_socio)

# Agrupar por id_socio y fecha_despacho, sumando cantidad_pallets_sacados
df_despacho_agrupado = df_despacho.groupby(['id_socio', 'fecha_despacho'], as_index=False).agg({
    'cantidad_pallets_sacados': 'sum',
    # Puedes agregar más columnas si es necesario
})

df_despacho_agrupado.head()


,id_socio,fecha_despacho,cantidad_pallets_sacados
0,1,2025-03-19,2
1,3,2025-01-04,4
2,3,2025-01-07,2
3,3,2025-01-09,5
4,3,2025-01-13,4


In [20]:
ruta = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/data'
df_despacho_agrupado.to_excel(f'{ruta}/despacho_limpio.xlsx', index=False)

In [21]:
# Filtrar las filas donde fecha_despacho corresponde al mes de marzo
df_marzo = df_despacho_agrupado[df_despacho_agrupado['fecha_despacho'].dt.month == 3]

df_marzo.head()

,id_socio,fecha_despacho,cantidad_pallets_sacados
0,1,2025-03-19,2
28,3,2025-03-01,3
29,3,2025-03-05,3
30,3,2025-03-07,2
31,3,2025-03-10,1


In [27]:
ruta = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/data'
df_marzo.to_excel(f'{ruta}/despacho_limpio_marzo1.xlsx', index=False)

In [24]:
print(df_despacho['fecha_despacho'].dtype)
print(df_despacho['fecha_despacho'].head())

datetime64[ns]
0   2025-01-02
1   2025-01-02
2   2025-01-02
3   2025-01-02
4   2025-01-03
Name: fecha_despacho, dtype: datetime64[ns]


In [25]:
df_despacho['fecha_despacho'] = pd.to_datetime(df_despacho['fecha_despacho']).dt.date


In [26]:
df_despacho['id_socio'] = df_despacho['id_socio'].astype(str)
map_socio = {str(k): v for k, v in map_socio.items()}
df_despacho['id_socio'] = df_despacho['id_socio'].replace(map_socio)
